In [8]:
import requests
import pandas

import datetime_translator

brandIds = { 202, 88, 31, 123, 101, 122, 36, 48, 135 }
data = {}
stationCount = {}

for brandId in brandIds:
    headers = { 'User-Agent': '', 'content-type':  'application/json' }
    jsonData = '{"variables":{"area":"salt-lake","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brandId name } latitude longitude id name prices(fuel: $fuel) { credit { nickname postedTime price  } fuelProduct } } } } }"}' %(brandId)
    #jsonData = '{"variables":{"area":"davis","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brandId name } latitude longitude id name prices(fuel: $fuel) { credit { nickname postedTime price  } fuelProduct } } } } }"}' %(brandId)
    #jsonData = '{"variables":{"area":"utah","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brandId name } latitude longitude id name prices(fuel: $fuel) { credit { nickname postedTime price  } fuelProduct } } } } }"}' %(brandId)
    response = requests.post('https://www.gasbuddy.com/graphql', headers=headers, data=jsonData)
    jsonResponse = response.json()
    stations = jsonResponse['data']['locationByArea']['stations']
    stationCount[brandId] = len(stations['results'])

    for station in stations['results']:
        if station['prices'][0]['credit']['price'] != 0:
            stationId = station['id']
            data[stationId] = {
                'StationName': station['name'],
                'BrandName': station['brands'][0]['name'],
                'AddressLine1': station['address']['line1'],
                'City': station['address']['locality'],
                'RegularFuelPrice': '${:.2f}'.format(station['prices'][0]['credit']['price']),
                'TimeSinceReported': datetime_translator.translate(station['prices'][0]['credit']['postedTime']),
                'ReportedBy': station['prices'][0]['credit']['nickname']
            }

output = pandas.DataFrame(data)
output.transpose().sort_values(by='RegularFuelPrice').head(20)

,StationName,BrandName,AddressLine1,City,RegularFuelPrice,TimeSinceReported,ReportedBy
62515,Maverik,Maverik,1680 South Redwood Road,Salt Lake City,$3.32,34 minutes,Owner
62566,Maverik,Maverik,1927 S. 3230 W.,Salt Lake City,$3.32,34 minutes,Owner
129536,Maverik,Maverik,425 South Redwood Road,Salt Lake City,$3.32,34 minutes,Owner
159136,Maverik,Maverik,1445 S. Gustin Road,Salt Lake City,$3.32,34 minutes,Owner
191646,Maverik,Maverik,1290 S. Wallace Road,SaltLake,$3.32,34 minutes,Owner
210515,Maverik,Maverik,2359 W California Ave,Salt Lake City,$3.32,33 minutes,Owner
201340,Maverik,Maverik,5969 West 4100 South,West Valley City,$3.49,34 minutes,Owner
62489,Maverik,Maverik,2707 W. 12600 S.,Riverton,$3.49,34 minutes,Owner
125730,Phillips 66,Phillips 66,8370 S State St,Midvale,$3.49,6 hours,brickford2012
62475,Shell,Shell,4489 Century Dr,Murray,$3.49,10 hours,sdenning80


In [9]:
stationCount

{36: 1, 101: 10, 135: 2, 202: 20, 48: 7, 88: 20, 122: 8, 123: 20, 31: 20}